In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.util import ngrams
import plotly.express as px
import scipy.stats
import math
import statistics
import string 
import re
from collections import defaultdict
from gensim import corpora, models

8 data (x) 
ranking negative(1-3) positive(4,5) (x) 
single words + Bi-grame + Tri-gram

### Import all the data 

In [2]:
# data importing 
lowes_6026 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3286026.csv')
lowes_0314 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3380314.csv')
lowes_0511 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_50280511.csv') 
lowes_6162 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_1003066162.csv')
lowes_6209 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5000026209.csv')
lowes_9855 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5001899855.csv')

In [3]:
# data importing 
homedepot_4796 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_204074796.csv')
homedepot_7071 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_319247071.csv')

In [4]:
# set display option to max to see all information in review column
pd.set_option('display.max_colwidth', -1)

C:\Users\lxf12\AppData\Local\Temp\ipykernel_21136\779606820.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [5]:
lowes_0314.head(2)

,Market_place,URL,Product_name,SKU,Price,Currency,Subject,Author,Date,Review,Stars,like,dislike
0,lowes.com,https://www.lowes.com/pd/American-Standard-Saver-White-WaterSense-Labeled-Elongated-Chair-Height-2-piece-Toilet-12-in-Rough-In-Size/3380314,American Standard Saver White Elongated Chair Height 2-piece WaterSense Toilet 12-in Rough-In Size (Ada Compliant),Model #710FA.101.020,179.0,USD,Need Quality Control,USAF157,"December 7, 2020",Nice design with poor quality control. Tank leaked during install. Had to replace fill valve and remove excess porcelain cement from fill valve opening to make it smooth and round.,1,0,0
1,lowes.com,https://www.lowes.com/pd/American-Standard-Saver-White-WaterSense-Labeled-Elongated-Chair-Height-2-piece-Toilet-12-in-Rough-In-Size/3380314,American Standard Saver White Elongated Chair Height 2-piece WaterSense Toilet 12-in Rough-In Size (Ada Compliant),Model #710FA.101.020,179.0,USD,Good Flush,EddieD,"April 12, 2013","I have bought other toilets for a rental. Sometimes there isn't much water in the bowl. Without enough water in bowl, you may need to clean the bowl much more often than usual, if you know what I mean. This toilet does hold a good amount of water in bowl. As far as people saying they need to flush 2 or 3 times....all you need to do is hold down the flushing handle for an extra 1-2 seconds and you get more water flushing from the tank through the bowl. (Quick flush for # 1 and Hold down lever longer for a long flush for #2). It does have a cheap seat, but I used the seat I had instead. The Wax Ring is good. It is deep for a good seal and it has the flange on it. Don't forget to get a very long water supply line. If not sure how long, just get the longest one they have. If too long, you can just twist it into a circle. You'll never see it behind the toilet. This is a high toilet (17 1/2 inches to top of seat). My wife is 5' 4"" . She said "" any higher she would need a step stool"" but she likes the height. We have been using this for over a month now with no complaints or concerns.",5,21,1


In [6]:
# drop null value
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]
namelist = ['lowes_6026', 'lowes_0314', 'lowes_0511', 'lowes_6162',
                    'lowes_6209', 'lowes_9855', 
                     'homedepot_4796', 'homedepot_7071']
n = 0 

for i in datalist: 
    # drop rows with Not a Number (NaN) and None values
    name = namelist[n]
    n += 1
    print(name + ' has null rows amount: '+ str(i['Review'].isna().sum()))
    i.dropna(subset=['Review'], inplace=True)
    print('after cleaning, ' + name + ' has null rows amount: ' 
          + str(i['Review'].isna().sum()))
    

lowes_6026 has null rows amount: 30
after cleaning, lowes_6026 has null rows amount: 0
lowes_0314 has null rows amount: 10
after cleaning, lowes_0314 has null rows amount: 0
lowes_0511 has null rows amount: 15
after cleaning, lowes_0511 has null rows amount: 0
lowes_6162 has null rows amount: 297
after cleaning, lowes_6162 has null rows amount: 0
lowes_6209 has null rows amount: 76
after cleaning, lowes_6209 has null rows amount: 0
lowes_9855 has null rows amount: 170
after cleaning, lowes_9855 has null rows amount: 0
homedepot_4796 has null rows amount: 186
after cleaning, homedepot_4796 has null rows amount: 0
homedepot_7071 has null rows amount: 9
after cleaning, homedepot_7071 has null rows amount: 0


In [7]:
lowes_9855.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 114
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Market_place  100 non-null    object 
 1   URL           100 non-null    object 
 2   Product_name  100 non-null    object 
 3   SKU           100 non-null    object 
 4   Price         100 non-null    float64
 5   Currency      100 non-null    object 
 6   Subject       100 non-null    object 
 7   Author        95 non-null     object 
 8   Date          100 non-null    object 
 9   Review        100 non-null    object 
 10  Stars         100 non-null    int64  
 11  like          100 non-null    int64  
 12  dislike       100 non-null    int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 10.9+ KB


In [8]:
datalist[0].isna().sum()

Market_place    0 
URL             0 
Product_name    0 
SKU             0 
Price           0 
Currency        0 
Subject         48
Author          11
Date            0 
Review          0 
Stars           0 
like            0 
dislike         0 
dtype: int64

### Separate Negative and Positive Ranking

In [9]:
# 8 data x 2 categories 
try:
    user_select = int(input("Please select negative(0) or positive ranking(1): "))
except: 
    print("Your input is not correct. Please input again according the tips.")

if user_select == 0:
    # get reviews that have given 1, 2, and 3 stars (negative)
    lowes_6026 = lowes_6026.loc[lowes_6026['Stars'] <= 3]
    lowes_0314 = lowes_0314.loc[lowes_0314['Stars'] <= 3]
    lowes_0511 = lowes_0511.loc[lowes_0511['Stars'] <= 3]
    lowes_6162 = lowes_6162.loc[lowes_6162['Stars'] <= 3]
    lowes_6209 = lowes_6209.loc[lowes_6209['Stars'] <= 3]
    lowes_9855 = lowes_9855.loc[lowes_9855['Stars'] <= 3]
    # get reviews that have given 1, 2, and 3 stars (negative)
    homedepot_4796 = homedepot_4796.loc[homedepot_4796['Stars'] <= 3]
    homedepot_7071 = homedepot_7071.loc[homedepot_7071['Stars'] <= 3]
elif user_select == 1:
    # get reviews that have given 4 and 5 stars (positive)
    lowes_6026 = lowes_6026.loc[lowes_6026['Stars'] > 3]
    lowes_0314 = lowes_0314.loc[lowes_0314['Stars'] > 3]
    lowes_0511 = lowes_0511.loc[lowes_0511['Stars'] > 3]
    lowes_6162 = lowes_6162.loc[lowes_6162['Stars'] > 3]
    lowes_6209 = lowes_6209.loc[lowes_6209['Stars'] > 3]
    lowes_9855 = lowes_9855.loc[lowes_9855['Stars'] > 3]
    # get reviews that have given 4 and 5 stars (positive)
    homedepot_4796 = homedepot_4796.loc[homedepot_4796['Stars'] > 3]
    homedepot_7071 = homedepot_7071.loc[homedepot_7071['Stars'] > 3]
else: 
    print("Your input is not correct. Please input again according the tips.")

Please select negative(0) or positive ranking(1): 1


In [10]:
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

In [11]:
lowes_0314.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 344 entries, 1 to 392
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Market_place  344 non-null    object 
 1   URL           344 non-null    object 
 2   Product_name  344 non-null    object 
 3   SKU           344 non-null    object 
 4   Price         344 non-null    float64
 5   Currency      344 non-null    object 
 6   Subject       335 non-null    object 
 7   Author        325 non-null    object 
 8   Date          344 non-null    object 
 9   Review        344 non-null    object 
 10  Stars         344 non-null    int64  
 11  like          344 non-null    int64  
 12  dislike       344 non-null    int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 37.6+ KB


### Lower case for all words & Remove punctuation

In [12]:
# download package punkt for word_tokenize
# nltk.download('punkt')
# download package for stopwords
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [13]:
# before the cleaning 
text_01 = lowes_6026['Review'].to_string(index=False)
# text_01

In [14]:
# Create a translation table with all punctuation characters mapped to None
translator = str.maketrans('', '', string.punctuation)

text = [0] * 8
n = 0 

for i in datalist:
    # convert pandas series data to string
    text[n] = i['Review'].to_string(index=False)
    # convert all characters to lowercase
    text[n] = text[n].lower()
    # remove punctuations using RegexpTokenizer
    text[n] = text[n].translate(translator)
    # Use regex to remove all numbers
    text[n] = re.sub(r'\d+', '', text[n])
    n += 1

In [15]:
# the result has been single words 
# text[0]

In [16]:
type(text[0])

str

### Stop words

In [17]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    # customize stop_words by including a custom list of words
#     custom_stop_words = ['new','toilet', 'factory', 'flush', 'sku', 'photos', 'stay', 
#                                      'lid', 'good', 'great', 'handle', 'top', 'high', 'urine']
#     stop_words.update(custom_stop_words)
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lxf12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
for i in range(8):
    text[i] = remove_stopwords(text[i])

In [19]:
# text[0]

In [20]:
# text[7]

### Text Tokenization

In [21]:
# remove punctuations using RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tokens = [0] * 8
for i in range(8):
    tokens[i] = tokenizer.tokenize(text[i])

In [22]:
# tokens[0]

### Bi-gram and Tri-gram

In [23]:
# N-gram generation
N = int(input('Which one do you want to look at? (0-7)'))

trigrams = [0] * 8
for i in range(8):
    trigrams[i] = list(ngrams(tokens[i], N))

Which one do you want to look at? (0-7)3


In [24]:
trigrams[6]

[('nice', 'color', 'sits'),
 ('color', 'sits', 'great'),
 ('sits', 'great', 'height'),
 ('great', 'height', 'getting'),
 ('height', 'getting', 'happy'),
 ('getting', 'happy', 'low'),
 ('happy', 'low', 'water'),
 ('low', 'water', 'levelbut'),
 ('water', 'levelbut', 'water'),
 ('levelbut', 'water', 'saving'),
 ('water', 'saving', 'thingso'),
 ('saving', 'thingso', 'good'),
 ('thingso', 'good', 'toileti'),
 ('good', 'toileti', 'didnt'),
 ('toileti', 'didnt', 'like'),
 ('didnt', 'like', 'plastic'),
 ('like', 'plastic', 'seat'),
 ('plastic', 'seat', 'changed'),
 ('seat', 'changed', 'one'),
 ('changed', 'one', 'see'),
 ('one', 'see', 'picture'),
 ('see', 'picture', 'toilet'),
 ('picture', 'toilet', 'great'),
 ('toilet', 'great', 'value'),
 ('great', 'value', 'looks'),
 ('value', 'looks', 'performs'),
 ('looks', 'performs', 'perfectly'),
 ('performs', 'perfectly', 'high'),
 ('perfectly', 'high', 'efficiency'),
 ('high', 'efficiency', 'quiet'),
 ('efficiency', 'quiet', 'attractive'),
 ('quiet'

In [25]:
# ## Count the trigrams with the same set of words
# trigram_count = defaultdict(set)

# for word in trigrams[6]:
#     key = frozenset(word)
#     trigram_count[key].add(word)

# # Print the trigram counts
# for key, amount in trigram_count.items():
#     count = len(amount)
#     print(f"{key}: {count}")

In [26]:
# connect the three words together 
words = [0] * 8
for i in range(8):
    words[i] = [' '.join(tr) for tr in trigrams[i]]

### Frequency Analysis

In [32]:
# select the data 
n = int(input('Which one do you want to look at? (0-7)'))

word_frequency = FreqDist(trigrams[n])

# Identify co-occurring trigrams
cooccurring_trigrams = []
for trigram in word_frequency:
    if word_frequency[trigram] > 1:
        cooccurring_trigrams.append(trigram)

word_frequency.most_common(10)

# word_frequency.plot(30,cumulative=False)
# plt.show()

Which one do you want to look at? (0-7)1


[(('water', 'supply', 'line'), 12),
 (('toilet', 'easy', 'install'), 8),
 (('american', 'standard', 'toilet'), 7),
 (('toilet', 'flushes', 'well'), 6),
 (('new', 'water', 'supply'), 4),
 (('would', 'highly', 'recommend'), 4),
 (('uses', 'less', 'water'), 4),
 (('easy', 'install', 'works'), 4),
 (('remove', 'old', 'toilet'), 3),
 (('one', 'master', 'bath'), 3)]

### Topic Modeling
#### Latent Dirichlet Allocation (LDA), which is a popular unsupervised learning algorithm used for topic modeling.

In [28]:
from gensim import corpora, models

In [33]:
# Step 1: select the data 
n = int(input('Which one do you want to look at? (0-7)'))

# Step 2: Create a dictionary of words and convert the text into a bag-of-words format
dictionary = corpora.Dictionary([words[n]])
corpus = [dictionary.doc2bow([word]) for word in words[n]]

# Step 3: Apply LDA to the corpus to identify topics in the text
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)

# Step 4: Print the topics and the top words associated with each topic
for i, topic in lda_model.show_topics(formatted=True, num_topics=3, num_words=4):
    print("Topic {}: {}".format(i, topic))

Which one do you want to look at? (0-7)1
Topic 0: 0.001*"toilet flushes well" + 0.001*"toilet easy install" + 0.001*"easy install works" + 0.001*"uses less water"
Topic 1: 0.001*"uses little water" + 0.001*"height elongated bowl" + 0.001*"flushes well clean" + 0.001*"right height elongated"
Topic 2: 0.001*"water supply line" + 0.001*"american standard toilet" + 0.001*"easy clean exterior" + 0.001*"american standard clean"
